# The epsilon-Greedy Algorithm

In computer science, a greedy algorithm is an algorithm that always takes whatever action seems best at the present moment, even when that decision might lead to bad long term consequences. The epsilon-Greedy algorithm is almost a greedy algorithm because it generally exploits the best available option, but every once in a while the epsilon-Greedy algorithm explores the other available options.

Using the previous example let's continue to optimise the colour of our web page. The epsilon-Greedy al‐ gorithm attempts to find the best color logo using the following procedure, which is applied to each new potential customer sequentially:
- When a new visitor comes to the site, the algorithm flips a coin that comes up tails with probability epsilon.
-  If the coin comes up heads, the algorithm is going to exploit. To exploit, the algo‐ rithm looks up the historical conversion rates for both the green and red logos in whatever data source it uses to keep track of things. After determining which color had the highest success rate in the past, the algorithm decides to show the new visitor the color that’s been most successful historically.
- If, instead of coming up heads, the coin comes up tails, the algorithm is going to explore. Since exploration involves randomly experimenting with the two colors being considered, the algorithm needs to flip a second coin to choose between them. Unlike the first coin, we’ll assume that this second coin comes up head 50% of the time.

After letting this algorithm loose on the visitors to a site for a long time, you’ll see that it works by oscillating between (A) exploiting the best option that it currently knows about and (B) exploring at random among all of the options available to it. In fact, you know from the definition of the algorithm that:
- With probability 1 – epsilon, the epsilon-Greedy algorithm exploits the best known option.
- With probability epsilon / 2, the epsilon-Greedy algorithm explores the best known option.
- With probability epsilon / 2, the epsilon-Greedy algorithm explores the worst known option.

## Describing Our Logo-Choosing Problem Abstractly
### What’s an Arm?
We want to consider the possibility that we have hundreds or thousands of colors to choose from, rather than just two. In general, we’re going to assume that we have a fixed set of N different options and that we can enumerate them, so that we can call our green logo "Option 1" and our red logo "Option 2" and any other logo "Option N". For historical reasons, these options are typically referred to as arms, so we’ll talk about "Arm 1" and "Arm 2" and "Arm N" rather than Option 1, Option 2 or Option N. But the main idea is the same regardless of the words we choose to employ.

### What’s a Reward?

Now that we’ve explained what an arm is, we’ve described one half of the abstract setup of the epsilon-Greedy algorithm. Next, we need to define a reward. A reward is simply a measure of success: it might tell us whether a customer clicked on an ad or signed up as a user. What matters is simply that (A) a reward is something quantitative that we can keep of track of mathematically and that (B) larger amounts of reward are better than smaller amounts.

### What’s a Bandit Problem?

Now that we’ve defined both arms and rewards, we can describe the abstract idea of a bandit problem that motivates all of the algorithms we’ll implement:
- We’re facing a complicated slot machine, called a bandit, that has a set of N arms that we can pull on.
- When pulled, any given arm will output a reward. But these rewards aren’t reliable, which is why we’re gambling: Arm 1 might give us 1 unit of reward only 1% of the time, while Arm 2 might give us 1 unit of reward only 3% of the time. Any specific pull of any specific arm is risky.
- Not only is each pull of an arm risky, we also don’t start off knowing what the reward rates are for any of the arms. We have to figure this out experimentally by actually pulling on the unknown arms.

So far the problem we’ve described in just a problem in statistics: you need to cope with risk by figuring out which arm has the highest average reward. You can calculate the average reward by pulling on each arm a lot of times and computing the mean of the rewards you get back. But a real bandit problem is more complicated and also more realistic.

What makes a bandit problem special is that we only receive a small amount of the information about the rewards from each arm. Specifically:
- We only find out about the reward that was given out by the arm we actually pulled. Whichever arm we pull, we miss out on information about the other arms that we didn’t pull. Just like in real life, you only learn about the path you took and not the paths you could have taken.

In fact, the situation is worse than that. Not only do we get only partial feedback about the wisdom of our past decisions, we’re literally falling behind every time we don’t make a good decision:
- Every time we experiment with an arm that isn’t the best arm, we lose reward be‐ cause we could, at least in principle, have pulled on a better arm.

The full Multiarmed Bandit Problem is defined by the five features above. Any algorithm that offers you a proposed solution to the Multiarmed Bandit Problem must give you a rule for selecting arms in some sequence. And this rule has to balance out your com‐ peting desires to (A) learn about new arms and (B) earn as much reward as possible by pulling on arms you already know are good choices.

## Implementing the epsilon-Greedy Algorithm

First, we define a class of objects that represents an epsilon-Greedy algorithm as it’s going to be deployed in the wild. This class will encapsulate the following pieces of information:

- **epsilon**: This will be a floating point number that tells us the frequency with which we should explore one of the available arms. If we set epsilon = 0.1, then we’ll explore the available arms on 10% of our pulls.
- **counts**: A vector of integers of length N that tells us how many times we’ve played each of the N arms available to us in the current bandit problem. If there are two arms, Arm 1 and Arm 2, which have both been played twice, then we’ll set counts = `[2, 2]`
- **values**: A vector of floating point numbers that defines the average amount of reward we’ve gotten when playing each of the N arms available to us. If Arm 1 gave us 1 unit of reward on one play and 0 on another play, while Arm 2 gave us 0 units of reward on both plays, then we’ll set values = `[0.5, 0.0]`

Put the above together in a constructor for a Python class:

In [1]:
class EpsilonGreedy:
    def __init__(self, epsilon, counts, values):
        self.epsilon = epsilon
        self.counts = counts
        self.values = values

Because the epsilon-Greedy algorithm’s behavior is very strongly controlled by the settings of both counts and values, we also provide explicit initialisation methods that let you reset these variables to their proper blank slate states before letting the algorithms loose:

In [3]:
def initialise(self, n_arms):
    self.counts = [0 for col in range(n_arms)]
    self.values = [0.0 for col in range(n_arms)]

The class becomes:

In [4]:
class EpsilonGreedy:
    def __init__(self, epsilon, counts, values):
        self.epsilon = epsilon
        self.counts = counts
        self.values = values
        
    def initialise(self, n_arms):
        self.counts = [0 for col in range(n_arms)]
        self.values = [0.0 for col in range(n_arms)]

Now that we have a class that represents all of the information that the epsilon-Greedy algorithm needs to keep track of about each of the arms, we need to define two types of behaviors that any algorithm for solving the Multiarmed Bandit Problem should provide:
- **select_arm**: Every time we have to make a choice about which arm to pull, we want to be able to simply make a call to our favorite algorithm and have it tell us the numeric name of the arm we should pull. Throughout this book, all of the bandit algorithms will implement a select_arm method that is called without any arguments and which returns the index of the next arm to pull.
- **update**: After we pull an arm, we get a reward signal back from our system. (In the next chapter, we’ll describe a testing framework we’ve built that simulates these rewards so that we can debug our bandit algorithms.) We want to update our algorithm’s beliefs about the quality of the arm we just chose by providing this reward information. Throughout this book, all of the bandit algorithms handle this by providing an update function that takes as arguments (1) an algorithm object, (2) the numeric index of the most recently chosen arm and (3) the reward received from choosing that arm. The update method will take this information and make the relevant changes to the algorithm’s evaluation of all of the arms.

Keeping in mind that general framework for behaviors that we expect a bandit algorithm to provide, let’s walk through the specific definition of these two functions for the epsilon-Greedy algorithm. First, we’ll implement `select_arm`:

In [6]:
def ind_max(x):
    m = max(x)
    return x.index(m) 

def select_arm(self):
    if random.random() > self.epsilon:
        return ind_max(self.values) 
    else:
        return random.randrange(len(self.values))

As you can see, the epsilon-Greedy algorithm handles selecting an arm in two parts: (1) we flip a coin to see if we’ll choose the best arm we know about and then (2) if the coin comes up tails, we’ll select an arm completely at random. In Python, we’ve implemented this by checking if a randomly generated number is greater than epsilon. If so, our algorithm selects the arm whose cached value according to the values field is highest; otherwise, it selects an arm at random.

These few lines of code completely describe the epsilon-Greedy algorithm’s solution to the Bandit problem: it explores some percentage of the time and otherwise chooses the arm it thinks is best. But, to understand which arm our epsilon-Greedy algorithm considers best, we need to define the update function. Let’s do that now, then explain why the procedure we’ve chosen is reasonable:

In [7]:
def update(self, chosen_arm, reward):
    self.counts[chosen_arm] = self.counts[chosen_arm] + 1
    n = self.counts[chosen_arm]
    value = self.values[chosen_arm]
    new_value = ((n - 1) / float(n)) * value + (1 / float(n)) * reward
    self.values[chosen_arm] = new_value

Looking at this code, we see that the update function first increments the counts field that records the number of times we’ve played each of the arms for this bandit problem to reflect the chosen arm. Then it finds the current estimated value of the chosen arm. If this is our first experience ever with the chosen arm, we set the estimated value directly to the reward we just received from playing that arm. **If we had played the arm in the past, we update the estimated value of the chosen arm to be a weighted average of the previously estimated value and the reward we just received. This weighting is important, because it means that single observations mean less and less to the algorithm when we already have a lot of experience with any specific option. The specific weighting we’ve chosen is designed to insure that the estimated value is exactly equal to the average of the rewards we’ve gotten from each arm.**

We suspect that it will not be obvious to many readers why this update rule computes a running average. To convince you why this works, consider the standard definition of an average:

In [8]:
def average(values):
    result = 0.0
    for value in values:
        result = result + value
    return result / len(values)

Instead of doing the division at the end, we could do it earlier on:

In [10]:
def running_average(values):
    result = 0.0
    n = float(len(values))
    for value in values:
        result = result + value / n
    return result

In [11]:
average([1,2,3,4,5])

3.0

In [12]:
running_average([1,2,3,4,5])

3.0

This alternative implementation looks much more like the update rule we’re using for the epsilon-Greedy algorithm. The core insight you need to have to fully see the rela‐ tionship between our update rule and this method for computing averages is this: the average of the first n – 1 values is just their sum divided by n – 1. So multiplying that average by (n – 1) / n will give you exactly the value that result has in the code above when you’ve processed the first n – 1 entries in values. If that explanation is not clear to you, we suggest that you print out the value of result at each step in the loop until you see the pattern we’re noting.

The final class becomes:

In [13]:
class EpsilonGreedy:
    def __init__(self, epsilon, counts, values):
        self.epsilon = epsilon
        self.counts = counts
        self.values = values
        
    def initialise(self, n_arms):
        self.counts = [0 for col in range(n_arms)]
        self.values = [0.0 for col in range(n_arms)]
        
    def ind_max(self, x):
        m = max(x)
        return x.index(m) 

    def select_arm(self):
        if random.random() > self.epsilon:
            return self.ind_max(self.values) 
        else:
            return random.randrange(len(self.values))
        
    def update(self, chosen_arm, reward):
        self.counts[chosen_arm] = self.counts[chosen_arm] + 1
        n = self.counts[chosen_arm]
        value = self.values[chosen_arm]
        new_value = ((n - 1) / float(n)) * value + (1 / float(n)) * reward
        self.values[chosen_arm] = new_value

## Thinking Critically about the epsilon-Greedy Algorithm

We can setup the epsilin-greedy algoritm to always select a random arm (explore) or to always select the best option (exploit). Setting the epsilon to `1.0` will always select a random arm. If you want to always select the best option we can set epsilon to `0.0` this way a random arm is never selected. These are the most extreme ways to setup the epsilon-greed algorithm. It's often better to select a middle ground where we both explore and exploit.

If we set epsilon = 1.0, the algorithm will always choose among the different arms completely at random. This will provide you with a lot of data, because all of the arms will have equal amounts of data and there won’t be any lurking confounds that make it hard to understand why you got the results you got. If you’re a traditionally trained scientist, this type of random experimentation will seem like a great approach. But it can lose a lot of money when you’re a business, because it means you’re as likely to try a bad idea as you are to try a good idea.

Or, to put the point another way: if you run a business, you should not want to accumulate a lot of data about bad options. You should want to only gather data about options that could plausibly be worth implementing. Setting epsilon = 1.0 wastes resources acquiring data about bad options.

If we eventually set epsilon = 0.0, it’s true that you’ll stop wasting time on bad options. But you’ll never be able to learn about new options ever again. If the world changes and you don’t provide your company with any mechanism for learning about the changes in the world, your com‐ pany will be left behind.

Thankfully, there’s no reason you need to operate at either of these two extremes. Instead of moving from one period of completely random experimentation to another period of absolutely greedy selection of the so-called best option, the epsilon-Greedy algorithm lets you be more gradual. You could, for example, set epsilon = 0.1 and leave the algorithm running forever.

But there are weaknesses with this approach as well. The first weakness is that, as you get more certain which of your amrs is best (out of a total of two arms), this tendency to explore the worse arm a full 5% of the time will become more wasteful. In the jargon of bandit algorithms, you’ll be over-exploring. And there’s another problem with a fixed 10% exploration rule: at the start of your experimentation, you’ll choose options that you don’t know much about far more rarely than you’d like to because you only try new options 10% of the time.